<a href="https://colab.research.google.com/github/bburaakk/BTK-Akademi-2025-Datathon/blob/main/datathon_2025_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

datathon_2025_path = kagglehub.competition_download('datathon-2025')

print('Data source import complete.')


# Kütüphaneler

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
import optuna
import warnings
warnings.filterwarnings('ignore')

print("📚 Tüm kütüphaneler yüklendi!")

# 1.VERİ YÜKLEME VE TEMEL TEMİZLEME

In [ ]:
# ===============================================================================
# 1. VERİ YÜKLEME VE TEMEL TEMİZLEME
# ===============================================================================

print("\n🔄 Veri yükleniyor...")
train = pd.read_csv("/kaggle/input/datathon-2025/train.csv")
test = pd.read_csv("/kaggle/input/datathon-2025/test.csv")

print(f"Train shape: {train.shape}, Test shape: {test.shape}")

# Duplicated temizleme
print("🧹 Duplicate veriler temizleniyor...")
initial_rows = train.shape[0]
train.drop_duplicates(inplace=True)
print(f"   {initial_rows - train.shape[0]} duplicate satır çıkarıldı")

# DateTime dönüşümü
print("📅 Datetime formatına çevriliyor...")
train['event_time'] = pd.to_datetime(train['event_time'])
test['event_time'] = pd.to_datetime(test['event_time'])

# Multi-user session temizleme
def clean_multi_user_sessions(df, df_name):
    session_user_counts = df.groupby('user_session')['user_id'].nunique()
    problematic_sessions = session_user_counts[session_user_counts > 1].index.tolist()

    if not problematic_sessions:
        print(f"   {df_name}: Sorunlu session yok")
        return df

    print(f"   {df_name}: {len(problematic_sessions)} sorunlu session düzeltiliyor")
    correction_map = {}
    for session_id in problematic_sessions:
        mode_user_id = df[df['user_session'] == session_id]['user_id'].mode()[0]
        correction_map[session_id] = mode_user_id

    for session_id, correct_user_id in correction_map.items():
        df.loc[df['user_session'] == session_id, 'user_id'] = correct_user_id

    return df

train = clean_multi_user_sessions(train, "Train")
test = clean_multi_user_sessions(test, "Test")

print("✅ Temel temizleme tamamlandı!")

# 2.DATA LEAK TESPİTİ VE AYIRMA

In [ ]:
# ===============================================================================
# 2. DATA LEAK TESPİTİ VE AYIRMA
# ===============================================================================

print("\n🕵️ Data leak tespiti...")
train_sessions = set(train['user_session'].unique())
test_sessions = set(test['user_session'].unique())
leaked_sessions = train_sessions.intersection(test_sessions)

print(f"   Leaked sessions: {len(leaked_sessions)}")

if len(leaked_sessions) > 0:
    # Leak map oluştur
    leaked_df_from_train = train[train['user_session'].isin(leaked_sessions)]
    leak_map_df = leaked_df_from_train[['user_session', 'session_value']].drop_duplicates()
    leak_map = pd.Series(leak_map_df.session_value.values,
                        index=leak_map_df.user_session).to_dict()

    # Test setini ayır
    test_leaked_df = test[test['user_session'].isin(leaked_sessions)].copy()
    test_unseen_df = test[~test['user_session'].isin(leaked_sessions)].copy()

    print(f"   Test leaked: {test_leaked_df.shape[0]}, Test unseen: {test_unseen_df.shape[0]}")
else:
    leak_map = {}
    test_unseen_df = test.copy()
    print("   Leak bulunamadı, tüm test verisi unseen olarak işleniyor")

# 3.GELİŞMİŞ FEATüRE ENGINEERİNG

In [ ]:
# ===============================================================================
# 3. GELİŞMİŞ FEATüRE ENGINEERİNG
# ===============================================================================

def advanced_feature_engineering(df_train, df_test):
    print("\n🚀 Gelişmiş Feature Engineering başlatılıyor...")

    # Combine datasets
    train_target = df_train[['user_session', 'session_value']].drop_duplicates()
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    df_full = pd.concat([df_train.drop('session_value', axis=1), df_test], ignore_index=True)

    # 1. TEMEL ÖZELLİKLER
    print("   📊 Temel özellikler...")
    session_df = df_full.groupby('user_session').agg(
        user_id=('user_id', 'first'),
        is_train=('is_train', 'first'),
        session_start_time=('event_time', 'min'),
        session_end_time=('event_time', 'max'),
        session_duration_seconds=('event_time', lambda x: (x.max() - x.min()).total_seconds()),
        total_events=('event_type', 'count'),
        unique_products=('product_id', 'nunique')
    ).reset_index()

    # 2. ZAMAN ÖZELLİKLERİ
    print("   ⏰ Zaman özellikleri...")
    session_df['start_hour'] = session_df['session_start_time'].dt.hour
    session_df['day_of_week'] = session_df['session_start_time'].dt.dayofweek
    session_df['is_weekend'] = (session_df['day_of_week'] >= 5).astype(int)
    session_df['start_month'] = session_df['session_start_time'].dt.month

    # Cyclical encoding
    session_df['hour_sin'] = np.sin(2 * np.pi * session_df['start_hour'] / 24)
    session_df['hour_cos'] = np.cos(2 * np.pi * session_df['start_hour'] / 24)
    session_df['day_sin'] = np.sin(2 * np.pi * session_df['day_of_week'] / 7)
    session_df['day_cos'] = np.cos(2 * np.pi * session_df['day_of_week'] / 7)

    # 3. EVENT TYPE FEATURES
    print("   🎯 Event type features...")
    event_counts = df_full.groupby('user_session')['event_type'].value_counts().unstack(fill_value=0)
    event_counts.columns = [f'{col}_count' for col in event_counts.columns]
    session_df = session_df.merge(event_counts, on='user_session', how='left')

    # Ensure all event types
    for event_type in ['VIEW', 'ADD_CART', 'REMOVE_CART', 'BUY']:
        if f'{event_type}_count' not in session_df.columns:
            session_df[f'{event_type}_count'] = 0

    # Behavioral ratios
    session_df['conversion_rate'] = session_df['BUY_count'] / (session_df['VIEW_count'] + 1)
    session_df['cart_abandonment_rate'] = session_df['REMOVE_CART_count'] / (session_df['ADD_CART_count'] + 1)
    session_df['add_to_cart_rate'] = session_df['ADD_CART_count'] / (session_df['VIEW_count'] + 1)
    session_df['purchase_efficiency'] = session_df['BUY_count'] / (session_df['total_events'] + 1)

    # 4. PRODUCT FEATURES
    print("   🛍️ Product features...")
    product_stats = df_full.groupby('user_session').agg(
        product_diversity=('product_id', lambda x: len(x.unique()) / len(x) if len(x) > 0 else 0),
        product_revisit_rate=('product_id', lambda x: 1 - len(x.unique()) / len(x) if len(x) > 0 else 0)
    )
    session_df = session_df.merge(product_stats, on='user_session', how='left')

    # 5. TF-IDF FEATURES (Simplified)
    print("   📝 TF-IDF features...")
    train_session_texts = df_full[df_full['is_train'] == 1].groupby('user_session')['product_id'].apply(
        lambda x: ' '.join(x.astype(str))
    )

    tfidf_vectorizer = TfidfVectorizer(max_features=200, min_df=2)
    tfidf_vectorizer.fit(train_session_texts)

    all_session_texts = df_full.groupby('user_session')['product_id'].apply(
        lambda x: ' '.join(x.astype(str))
    )
    tfidf_matrix = tfidf_vectorizer.transform(all_session_texts)

    # PCA for dimensionality reduction
    pca = PCA(n_components=30, random_state=42)
    tfidf_pca = pca.fit_transform(tfidf_matrix.toarray())

    tfidf_df = pd.DataFrame(tfidf_pca,
                           index=all_session_texts.index,
                           columns=[f'prod_pca_{i}' for i in range(30)])
    session_df = session_df.merge(tfidf_df, on='user_session', how='left')

    # 6. SEQUENCE FEATURES
    print("   🔄 Sequence features...")
    def get_first_last_events(group):
        events = group.sort_values('event_time')
        return pd.Series({
            'first_event': events['event_type'].iloc[0] if len(events) > 0 else 'VIEW',
            'last_event': events['event_type'].iloc[-1] if len(events) > 0 else 'VIEW',
            'event_diversity': len(events['event_type'].unique())
        })

    sequence_features = df_full.groupby('user_session').apply(get_first_last_events)
    session_df = session_df.merge(sequence_features, on='user_session', how='left')

    # 7. ENCODING - Düzeltilmiş
    print("   🏷️ Encoding (Düzeltilmiş)...")

    # Split data back before encoding
    train_session_df = session_df[session_df['is_train'] == 1].copy()
    test_session_df = session_df[session_df['is_train'] == 0].copy()

    # Fit encoders ONLY on train data
    le_user = LabelEncoder()
    train_session_df['user_id_encoded'] = le_user.fit_transform(train_session_df['user_id'])

    le_first = LabelEncoder()
    train_session_df['first_event_encoded'] = le_first.fit_transform(train_session_df['first_event'])

    le_last = LabelEncoder()
    train_session_df['last_event_encoded'] = le_last.fit_transform(train_session_df['last_event'])

    # Transform test data - handle unseen categories
    # For unseen categories in test, assign a value that will be treated as missing or a separate category by models
    # Here, we will use -1 for simplicity, assuming models can handle it or we will one-hot encode later
    # A more robust approach would involve mapping to a common 'unknown' category if one-hot encoding
    def transform_with_unseen(le, series):
        return series.map(lambda s: le.transform([s])[0] if s in le.classes_ else -1)

    test_session_df['user_id_encoded'] = transform_with_unseen(le_user, test_session_df['user_id'])
    test_session_df['first_event_encoded'] = transform_with_unseen(le_first, test_session_df['first_event'])
    test_session_df['last_event_encoded'] = transform_with_unseen(le_last, test_session_df['last_event'])


    # Duration segments (can be done on combined data as it's value-based)
    session_df['duration_log'] = np.log1p(session_df['session_duration_seconds'])
    session_df['duration_segment'] = pd.cut(session_df['session_duration_seconds'],
                                           bins=[0, 60, 300, 1800, 7200, float('inf')],
                                           labels=['Very_Short', 'Short', 'Medium', 'Long', 'Very_Long'],
                                           right=False) # Use right=False for intervals [a, b)

    # Merge duration features back
    duration_features = session_df[['user_session', 'duration_log', 'duration_segment']]
    train_session_df = train_session_df.merge(duration_features, on='user_session', how='left')
    test_session_df = test_session_df.merge(duration_features, on='user_session', how='left')


    # 8. CLEANUP
    print("   🧹 Temizlik...")
    # Drop original categorical columns and is_train
    cols_to_drop = ['user_id', 'session_start_time', 'session_end_time',
                    'first_event', 'last_event', 'is_train']

    final_train_df = train_session_df.drop(cols_to_drop, axis=1)
    final_test_df = test_session_df.drop(cols_to_drop, axis=1)

    # Fill missing values (should be minimal after encoding fix, but keep as safety)
    # Categorical columns with potential -1 from unseen should be handled by model or one-hot encoding
    # Numeric columns should be filled, e.g., with 0 or mean/median of train data
    numeric_cols_train = final_train_df.select_dtypes(include=np.number).columns
    numeric_cols_test = final_test_df.select_dtypes(include=np.number).columns

    # Fill numeric NaNs with 0 (or consider a better strategy like train mean/median)
    final_train_df[numeric_cols_train] = final_train_df[numeric_cols_train].fillna(0)
    final_test_df[numeric_cols_test] = final_test_df[numeric_cols_test].fillna(0)

    # Handle 'duration_segment' - fill NaNs with a default category if any
    if 'duration_segment' in final_train_df.columns:
         final_train_df['duration_segment'] = final_train_df['duration_segment'].cat.add_categories('Unknown').fillna('Unknown')
         final_test_df['duration_segment'] = final_test_df['duration_segment'].cat.add_categories('Unknown').fillna('Unknown')


    # Merge target back to train
    final_train_df = final_train_df.merge(train_target, on='user_session', how='left')

    print(f"   ✅ Feature engineering tamamlandı! Train: {final_train_df.shape}, Test: {final_test_df.shape}")
    return final_train_df, final_test_df

# Feature engineering çalıştır
train_features_df, test_features_df = advanced_feature_engineering(train, test_unseen_df)

In [ ]:
display(train_features_df.columns)

# 4.MODEL EĞİTİMİ (ENHANCED ENSEMBLE)

In [ ]:
# ===============================================================================
# 4. MODEL EĞİTİMİ (ENHANCeD ENSEMBLE)
# ===============================================================================

print("\n🤖 Model eğitimi başlatılıyor...")

# Veriyi hazırla
y_train = train_features_df['session_value']
y_train_log = np.log1p(y_train)
X_train = train_features_df.drop(['user_session', 'session_value'], axis=1)
X_test = test_features_df.drop(['user_session'], axis=1)

# Sütunları hizala
X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)

print(f"   📊 X_train: {X_train.shape}, X_test: {X_test_aligned.shape}")

# Train-validation split
X_train_part, X_val, y_train_part, y_val = train_test_split(
    X_train, y_train_log, test_size=0.2, random_state=42
)

# 5.QUICK ENSEMBLE (3 MODEL)

In [ ]:
# ===============================================================================
# 5. QUICK ENSEMBLE (3 MODEL)
# ===============================================================================

print("\n🎯 3-Model Ensemble eğitiliyor...")

models = {}
predictions = {}

# Kategorik sütunları tespit et ve dönüştür
categorical_features = []
cat_indices = []

# X_train'in bir kopyasını al (LightGBM için)
X_train_lgb = X_train_part.copy()
X_val_lgb = X_val.copy()

for idx, col in enumerate(X_train.columns):
    if X_train[col].dtype == 'category' or col.endswith('_encoded') or col == 'duration_segment':
        categorical_features.append(col)
        cat_indices.append(idx)

        # LightGBM için kategorik sütunları int'e çevir
        if X_train_lgb[col].dtype == 'object' or X_train_lgb[col].dtype == 'category':
            # Önce string'e sonra int'e çevir
            X_train_lgb[col] = X_train_lgb[col].astype(str).astype('category').cat.codes
            X_val_lgb[col] = X_val_lgb[col].astype(str).astype('category').cat.codes

print(f"   🏷️ Tespit edilen kategorik sütunlar: {categorical_features}")
print(f"   📊 Kategorik sütun indeksleri: {cat_indices}")

# 1. CatBoost
print("\n   🐱 CatBoost eğitiliyor...")
cat_params = {
    'iterations': 2000,
    'learning_rate': 0.05,
    'depth': 8,
    'l2_leaf_reg': 3.0,
    'random_strength': 1.0,
    'bagging_temperature': 0.2,
    'verbose': 0,
    'random_seed': 42,
    'task_type': 'GPU' if 'GPU' in str(globals().get('device', '')) else 'CPU'
}

models['catboost'] = CatBoostRegressor(**cat_params)
models['catboost'].fit(X_train_part, y_train_part,
                      eval_set=[(X_val, y_val)],
                      cat_features=categorical_features,
                      early_stopping_rounds=100,
                      verbose=0)

val_pred_cat = models['catboost'].predict(X_val)
rmse_cat = np.sqrt(mean_squared_error(y_val, val_pred_cat))
print(f"      ✅ Validation RMSE: {rmse_cat:.6f}")

# 2. LightGBM
print("\n   💡 LightGBM eğitiliyor...")
lgb_params = {
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,
    'verbose': -1,
    'n_estimators': 2000
}

# LightGBM için kategorik indeksleri belirt
models['lightgbm'] = lgb.LGBMRegressor(**lgb_params)
models['lightgbm'].fit(
    X_train_lgb, y_train_part,
    eval_set=[(X_val_lgb, y_val)],
    categorical_feature=cat_indices,  # Kategorik özellik indekslerini belirt
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
)

val_pred_lgb = models['lightgbm'].predict(X_val_lgb)
rmse_lgb = np.sqrt(mean_squared_error(y_val, val_pred_lgb))
print(f"      ✅ Validation RMSE: {rmse_lgb:.6f}")

# 3. XGBoost
print("\n   🚀 XGBoost eğitiliyor...")
xgb_params = {
    'n_estimators': 2000,
    'learning_rate': 0.05,
    'max_depth': 6,
    'min_child_weight': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,
    'tree_method': 'hist',  # Daha hızlı eğitim için
    'enable_categorical': True,  # XGBoost'ta kategorik destek
    'early_stopping_rounds': 100  # Yeni versiyonda buraya taşındı
}

# XGBoost için de kategorik sütunları dönüştür
X_train_xgb = X_train_part.copy()
X_val_xgb = X_val.copy()

for col in categorical_features:
    if X_train_xgb[col].dtype == 'object':
        X_train_xgb[col] = X_train_xgb[col].astype('category')
        X_val_xgb[col] = X_val_xgb[col].astype('category')

models['xgboost'] = xgb.XGBRegressor(**xgb_params)
models['xgboost'].fit(
    X_train_xgb, y_train_part,
    eval_set=[(X_val_xgb, y_val)],
    verbose=0
)

val_pred_xgb = models['xgboost'].predict(X_val_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_val, val_pred_xgb))
print(f"      ✅ Validation RMSE: {rmse_xgb:.6f}")

# Ensemble tahminleri - ağırlıklı ortalama
print("\n   🔄 Ensemble tahminleri hesaplanıyor...")

# Performansa göre dinamik ağırlıklar
total_inv_rmse = 1/rmse_cat + 1/rmse_lgb + 1/rmse_xgb
weights = {
    'catboost': (1/rmse_cat) / total_inv_rmse,
    'lightgbm': (1/rmse_lgb) / total_inv_rmse,
    'xgboost': (1/rmse_xgb) / total_inv_rmse
}

print(f"\n   📊 Model Ağırlıkları:")
print(f"      CatBoost: {weights['catboost']:.3f}")
print(f"      LightGBM: {weights['lightgbm']:.3f}")
print(f"      XGBoost:  {weights['xgboost']:.3f}")

# Ağırlıklı ensemble
ensemble_val_pred = (
    weights['catboost'] * val_pred_cat +
    weights['lightgbm'] * val_pred_lgb +
    weights['xgboost'] * val_pred_xgb
)

ensemble_rmse = np.sqrt(mean_squared_error(y_val, ensemble_val_pred))
print(f"\n   🎯 Ensemble Validation RMSE: {ensemble_rmse:.6f}")

# Model performans özeti
print("\n" + "="*60)
print("   📈 MODEL PERFORMANS ÖZETİ")
print("="*60)
print(f"   CatBoost RMSE:  {rmse_cat:.6f}")
print(f"   LightGBM RMSE:  {rmse_lgb:.6f}")
print(f"   XGBoost RMSE:   {rmse_xgb:.6f}")
print(f"   Ensemble RMSE:  {ensemble_rmse:.6f}")
print(f"   İyileştirme:    {min(rmse_cat, rmse_lgb, rmse_xgb) - ensemble_rmse:.6f}")
print("="*60)

# 6.FINAL MODEL EĞİTİMİ (FULL DATA - 2 MODEL)

In [ ]:
# ===============================================================================
# 6. FINAL MODEL EĞİTİMİ (FULL DATA - 2 MODEL)
# ===============================================================================

print("\n🏆 Final model eğitimi (full data - 2 model)...")

final_models = {}

# Kategorik sütunları belirle
categorical_features = []
for col in X_train.columns:
    if X_train[col].dtype == 'category':
        categorical_features.append(col)

print(f"📊 Kategorik sütunlar: {categorical_features}")

# CatBoost final
final_models['catboost'] = CatBoostRegressor(**cat_params)
final_models['catboost'].fit(
    X_train,
    y_train_log,
    cat_features=categorical_features,
    verbose=0
)

# LightGBM final - mevcut parametreleri kullan (zaten n_estimators=2000 var)
final_models['lightgbm'] = lgb.LGBMRegressor(**lgb_params)
final_models['lightgbm'].fit(X_train, y_train_log)

print("✅ Final model eğitimi tamamlandı!")

# Model performanslarını kontrol et
print("\n📈 Final model performansları:")
for name, model in final_models.items():
    train_pred = model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(y_train_log, train_pred))
    print(f"{name.upper()}: Train RMSE = {train_rmse:.4f}")

# 7.TAHMİN VE SUBMİSSİON (2 MODEL)

In [ ]:
# ===============================================================================
# 7. TAHMİN VE SUBMİSSİON (2 MODEL)
# ===============================================================================

print("\n🔮 Tahminler yapılıyor...")

# Test predictions
test_preds = {}
for name, model in final_models.items():
    test_preds[name] = model.predict(X_test_aligned)

# Ensemble prediction (adjusted for 2 models)
final_prediction_log = (weights['catboost'] * test_preds['catboost'] +
                       weights['lightgbm'] * test_preds['lightgbm'])

# Convert back to original scale
final_prediction = np.expm1(final_prediction_log)
final_prediction[final_prediction < 0] = 0

# Create model predictions DataFrame
model_predictions_df = pd.DataFrame({
    'user_session': test_features_df['user_session'],
    'session_value': final_prediction
})

# 8.SUBMİSSİON DOSYASI OLUŞTURMA

In [ ]:
# ===============================================================================
# 8. SUBMİSSİON DOSYASI OLUŞTURMA
# ===============================================================================

print("\n📝 Submission dosyası oluşturuluyor...")

# Official template
submission_df = test[['user_session']].drop_duplicates().copy()

# Fill leaked data first
if len(leak_map) > 0:
    submission_df['session_value'] = submission_df['user_session'].map(leak_map)
    leaked_count = len(submission_df) - submission_df['session_value'].isnull().sum()
    print(f"   🔍 {leaked_count} leaked prediction eklendi")
else:
    submission_df['session_value'] = np.nan
    leaked_count = 0

# Fill remaining with model predictions
model_predictions_map = model_predictions_df.set_index('user_session')['session_value']
submission_df['session_value'] = submission_df['session_value'].fillna(
    submission_df['user_session'].map(model_predictions_map)
)

filled_by_model = len(submission_df) - submission_df['session_value'].isnull().sum() - leaked_count
print(f"   🤖 {filled_by_model} model prediction eklendi")

# Final check
if submission_df['session_value'].isnull().sum() > 0:
    print("   ⚠️ Boş değerler ortalama ile dolduruluyor...")
    submission_df['session_value'].fillna(y_train.mean(), inplace=True)

# Save submission
submission_path = '/content/improved_submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"🎉 BAŞARILI! Submission dosyası hazır: {submission_path}")
print(f"📊 Toplam prediction: {len(submission_df)}")
print(f"📈 Ensemble validation RMSE: {ensemble_rmse:.6f}")
print("\n" + "="*60)
print("🏁 İyileştirilmiş model pipeline tamamlandı!")
print("="*60)# 1.VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN)

# 1.VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN)

In [ ]:
# ===============================================================================
# 1. VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN)
# ===============================================================================

print("\n🔄 Adversarial validation için veri hazırlanıyor...")

# Train ve test veri setlerine kaynak etiketi ekle
X_train['source'] = 0  # 0 for train
X_test_aligned['source'] = 1 # 1 for test

# Train ve test özelliklerini birleştir
# Kategorik sütunları birleştirmeden önce object'e çevir
for col in X_train.columns:
    if X_train[col].dtype == 'category':
        X_train[col] = X_train[col].astype(object)
        X_test_aligned[col] = X_test_aligned[col].astype(object)


adversarial_data = pd.concat([X_train, X_test_aligned], ignore_index=True)

# Hedef değişkeni (source) ayır
adversarial_target = adversarial_data['source']
adversarial_features = adversarial_data.drop('source', axis=1)

print(f"   Birleştirilmiş veri boyutu: {adversarial_data.shape}")
print("✅ Veri hazırlığı tamamlandı!")

# 2.ADVERSARIAL MODEL EĞİTİMİ (LightGBM İLE)

In [ ]:
# ===============================================================================
# 2. ADVERSARIAL MODEL EĞİTİMİ
# ===============================================================================

print("\n adversarial model eğitiliyor...")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Adversarial model için LightGBM sınıflandırıcısı
lgb_adv_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'seed': 42,
    'n_jobs': -1,
    'verbose': -1,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
}

# Kategorik özellikleri tekrar belirle (birleştirilmiş veri üzerinde)
# Birleştirmeden önce object'e çevrildikleri için burada tekrar category yapalım
for col in adversarial_features.columns:
    if adversarial_features[col].dtype == 'object':
        adversarial_features[col] = adversarial_features[col].astype('category')

categorical_features_adv = [col for col in adversarial_features.columns if adversarial_features[col].dtype == 'category']

print(f"   Adversarial model için kategorik sütunlar: {categorical_features_adv}")


# Çapraz doğrulama ile eğit
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(adversarial_data.shape[0])
feature_importances = pd.DataFrame(index=adversarial_features.columns)

for fold, (train_idx, val_idx) in enumerate(kf.split(adversarial_features, adversarial_target)):
    print(f"\n   Fold {fold+1}/5")
    X_train_fold, X_val_fold = adversarial_features.iloc[train_idx], adversarial_features.iloc[val_idx]
    y_train_fold, y_val_fold = adversarial_target.iloc[train_idx], adversarial_target.iloc[val_idx]

    model = lgb.LGBMClassifier(**lgb_adv_params)

    model.fit(X_train_fold, y_train_fold,
              eval_set=[(X_val_fold, y_val_fold)],
              eval_metric='auc',
              callbacks=[lgb.early_stopping(100, verbose=False)],
              categorical_feature=categorical_features_adv)

    oof_preds[val_idx] = model.predict_proba(X_val_fold)[:, 1]
    feature_importances[f'fold_{fold+1}'] = model.feature_importances_

# OOF tahminleri üzerinden AUC hesapla
overall_auc = roc_auc_score(adversarial_target, oof_preds)

print(f"\n✅ Adversarial model eğitimi tamamlandı!")
print(f"   Overall Adversarial AUC: {overall_auc:.6f}")

# Feature importances ortalamasını hesapla
feature_importances['average'] = feature_importances.mean(axis=1)
feature_importances = feature_importances.sort_values('average', ascending=False)

print("\n   Önemli Özellikler (En Yüksekten En Düşüğe):")
print(feature_importances[['average']].head(20)) # En önemli 20 özelliği göster

# 2.ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE)

In [ ]:
# ===============================================================================
# 2. ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE - YENİ DENEME)
# ===============================================================================

print("\n adversarial model eğitiliyor (CatBoost)...")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# Adversarial model için CatBoost sınıflandırıcısı
cat_adv_params = {
    'objective': 'Logloss',  # Binary classification objective
    'eval_metric': 'AUC',
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'random_seed': 42,
    'verbose': 0, # Daha fazla bilgi için 100 yapabilirsiniz
    'task_type': 'GPU' if 'GPU' in str(globals().get('device', '')) else 'CPU',
    'early_stopping_rounds': 100
}

# Kategorik özellikleri belirle
# CatBoost string/object türündeki kategoriklere doğrudan destek verir.
# Önceki adımda object'e çevirmiştik, CatBoost bunu kabul eder.
categorical_features_adv = [col for col in adversarial_features.columns if adversarial_features[col].dtype == 'object' or adversarial_features[col].dtype == 'category']

print(f"   Adversarial model için kategorik sütunlar: {categorical_features_adv}")


# Çapraz doğrulama ile eğit
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds_catboost = np.zeros(adversarial_data.shape[0])
feature_importances_catboost = pd.DataFrame(index=adversarial_features.columns)

for fold, (train_idx, val_idx) in enumerate(kf.split(adversarial_features, adversarial_target)):
    print(f"\n   Fold {fold+1}/5")
    X_train_fold, X_val_fold = adversarial_features.iloc[train_idx], adversarial_features.iloc[val_idx]
    y_train_fold, y_val_fold = adversarial_target.iloc[train_idx], adversarial_target.iloc[val_idx]

    model = CatBoostClassifier(**cat_adv_params)

    model.fit(X_train_fold, y_train_fold,
              eval_set=[(X_val_fold, y_val_fold)],
              cat_features=categorical_features_adv,
              early_stopping_rounds=cat_adv_params['early_stopping_rounds'],
              verbose=cat_adv_params['verbose'])

    oof_preds_catboost[val_idx] = model.predict_proba(X_val_fold)[:, 1]
    feature_importances_catboost[f'fold_{fold+1}'] = model.get_feature_importance() # CatBoost feature importance alma

# OOF tahminleri üzerinden AUC hesapla
overall_auc_catboost = roc_auc_score(adversarial_target, oof_preds_catboost)

print(f"\n✅ Adversarial model eğitimi tamamlandı!")
print(f"   Overall Adversarial AUC (CatBoost): {overall_auc_catboost:.6f}")

# Feature importances ortalamasını hesapla
feature_importances_catboost['average'] = feature_importances_catboost.mean(axis=1)
feature_importances_catboost = feature_importances_catboost.sort_values('average', ascending=False)

print("\n   Önemli Özellikler (En Yüksekten En Düşüğe - CatBoost):")
print(feature_importances_catboost[['average']].head(20)) # En önemli 20 özelliği göster

# 3.ÖNEMLİ ÖZELLİKLERİN DAĞILIMLARINI GÖRSELLEŞTİRME





In [ ]:
# ===============================================================================
# 3. ÖNEMLİ ÖZELLİKLERİN DAĞILIMLARINI GÖRSELLEŞTİRME
# ===============================================================================

print("\n📊 En önemli özelliklerin dağılımları görselleştiriliyor...")

# Adversarial modelin en önemli bulduğu özellikler (ilk 10)
top_features = feature_importances_catboost['average'].head(10).index.tolist()
print(f"   Görselleştirilecek özellikler: {top_features}")

# Adversarial data frame'ini tekrar train ve test olarak ayır
adversarial_data['source'] = adversarial_target # Add source back for splitting
train_adversarial = adversarial_data[adversarial_data['source'] == 0].drop('source', axis=1)
test_adversarial = adversarial_data[adversarial_data['source'] == 1].drop('source', axis=1)


# Dağılımları görselleştir
plt.figure(figsize=(15, 10))
for i, feature in enumerate(top_features):
    plt.subplot(4, 3, i + 1) # Adjust grid size based on number of features
    sns.histplot(data=train_adversarial, x=feature, color='skyblue', label='Train', kde=True, stat='density', common_norm=False)
    sns.histplot(data=test_adversarial, x=feature, color='lightcoral', label='Test', kde=True, stat='density', common_norm=False)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()

plt.tight_layout()
plt.show()

print("✅ Görselleştirmeler tamamlandı!")

# 4.ADVERSARIAL MODEL FEATURE IMPORTANCE GÖRSELLEŞTİRME

In [ ]:
# ===============================================================================
# 4. ADVERSARIAL MODEL FEATURE IMPORTANCE GÖRSELLEŞTİRME
# ===============================================================================

print("\n📊 Adversarial model feature importance görselleştiriliyor...")

# CatBoost modelinden elde edilen ortalama feature importance değerleri
feature_importance = feature_importances_catboost['average']

# Feature importance değerlerini sırala
sorted_idx = feature_importance.sort_values(ascending=False).index
sorted_importance = feature_importance[sorted_idx]

# Görselleştirmeyi yap
plt.figure(figsize=(10, 8))
plt.barh(range(len(sorted_idx)), sorted_importance, align='center')
plt.yticks(range(len(sorted_idx)), sorted_idx)
plt.xlabel('Average Feature Importance (Adversarial)')
plt.title('Adversarial Model Feature Importance')
plt.gca().invert_yaxis() # En önemli özelliği en üste almak için y eksenini ters çevir

plt.tight_layout()
plt.show()

print("✅ Feature importance görselleştirme tamamlandı!")

# 1.VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN) - user_id_encoded çıkarıldı

In [ ]:
# ===============================================================================
# 1. VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN) - user_id_encoded çıkarıldı
# ===============================================================================

print("\n🔄 Adversarial validation için veri hazırlanıyor (user_id_encoded çıkarıldı)...")

# Train ve test veri setlerine kaynak etiketi ekle
X_train_modified = X_train.copy() # Kopya al
X_test_aligned_modified = X_test_aligned.copy() # Kopya al

# user_id_encoded sütununu çıkar
if 'user_id_encoded' in X_train_modified.columns:
    X_train_modified = X_train_modified.drop('user_id_encoded', axis=1)
    print("   'user_id_encoded' train özelliklerinden çıkarıldı.")
if 'user_id_encoded' in X_test_aligned_modified.columns:
    X_test_aligned_modified = X_test_aligned_modified.drop('user_id_encoded', axis=1)
    print("   'user_id_encoded' test özelliklerinden çıkarıldı.")


X_train_modified['source'] = 0  # 0 for train
X_test_aligned_modified['source'] = 1 # 1 for test

# Train ve test özelliklerini birleştir
# Kategorik sütunları birleştirmeden önce object'e çevir
for col in X_train_modified.columns:
    if X_train_modified[col].dtype == 'category':
        X_train_modified[col] = X_train_modified[col].astype(object)
        X_test_aligned_modified[col] = X_test_aligned_modified[col].astype(object)


adversarial_data_no_userid = pd.concat([X_train_modified, X_test_aligned_modified], ignore_index=True)

# Hedef değişkeni (source) ayır
adversarial_target_no_userid = adversarial_data_no_userid['source']
adversarial_features_no_userid = adversarial_data_no_userid.drop('source', axis=1)

print(f"   Birleştirilmiş veri boyutu (user_id_encoded olmadan): {adversarial_data_no_userid.shape}")
print("✅ Veri hazırlığı tamamlandı!")

# 2.ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE - user_id_encoded olmadan)

In [ ]:
# ===============================================================================
# 2. ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE - user_id_encoded olmadan)
# ===============================================================================

print("\n adversarial model eğitiliyor (CatBoost - user_id_encoded olmadan)...")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# Adversarial model için CatBoost sınıflandırıcısı - Parametreler aynı kalabilir
cat_adv_params_no_userid = {
    'objective': 'Logloss',  # Binary classification objective
    'eval_metric': 'AUC',
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'random_seed': 42,
    'verbose': 0, # Daha fazla bilgi için 100 yapabilirsiniz
    'task_type': 'GPU' if 'GPU' in str(globals().get('device', '')) else 'CPU',
    'early_stopping_rounds': 100
}

# Kategorik özellikleri belirle (user_id_encoded olmadan)
# CatBoost string/object türündeki kategoriklere doğrudan destek verir.
categorical_features_adv_no_userid = [col for col in adversarial_features_no_userid.columns if adversarial_features_no_userid[col].dtype == 'object' or adversarial_features_no_userid[col].dtype == 'category']

print(f"   Adversarial model için kategorik sütunlar (user_id_encoded olmadan): {categorical_features_adv_no_userid}")


# Çapraz doğrulama ile eğit
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds_catboost_no_userid = np.zeros(adversarial_data_no_userid.shape[0])
feature_importances_catboost_no_userid = pd.DataFrame(index=adversarial_features_no_userid.columns)

for fold, (train_idx, val_idx) in enumerate(kf.split(adversarial_features_no_userid, adversarial_target_no_userid)):
    print(f"\n   Fold {fold+1}/5")
    X_train_fold, X_val_fold = adversarial_features_no_userid.iloc[train_idx], adversarial_features_no_userid.iloc[val_idx]
    y_train_fold, y_val_fold = adversarial_target_no_userid.iloc[train_idx], adversarial_target_no_userid.iloc[val_idx]

    model = CatBoostClassifier(**cat_adv_params_no_userid)

    model.fit(X_train_fold, y_train_fold,
              eval_set=[(X_val_fold, y_val_fold)],
              cat_features=categorical_features_adv_no_userid,
              early_stopping_rounds=cat_adv_params_no_userid['early_stopping_rounds'],
              verbose=cat_adv_params_no_userid['verbose'])

    oof_preds_catboost_no_userid[val_idx] = model.predict_proba(X_val_fold)[:, 1]
    feature_importances_catboost_no_userid[f'fold_{fold+1}'] = model.get_feature_importance() # CatBoost feature importance alma

# OOF tahminleri üzerinden AUC hesapla
overall_auc_catboost_no_userid = roc_auc_score(adversarial_target_no_userid, oof_preds_catboost_no_userid)

print(f"\n✅ Adversarial model eğitimi tamamlandı!")
print(f"   Overall Adversarial AUC (CatBoost - user_id_encoded olmadan): {overall_auc_catboost_no_userid:.6f}")

# Feature importances ortalamasını hesapla
feature_importances_catboost_no_userid['average'] = feature_importances_catboost_no_userid.mean(axis=1)
feature_importances_catboost_no_userid = feature_importances_catboost_no_userid.sort_values('average', ascending=False)

print("\n   Önemli Özellikler (En Yüksekten En Düşüğe - CatBoost - user_id_encoded olmadan):")
print(feature_importances_catboost_no_userid[['average']].head(20)) # En önemli 20 özelliği göster

# 4.ADVERSARIAL MODEL FEATURE IMPORTANCE GÖRSELLEŞTİRME (USER_ID_ENCODED OLMADAN)

In [ ]:
# ===============================================================================
# 4. ADVERSARIAL MODEL FEATURE IMPORTANCE GÖRSELLEŞTİRME (USER_ID_ENCODED OLMADAN)
# ===============================================================================

print("\n📊 Adversarial model feature importance görselleştiriliyor (user_id_encoded olmadan)...")

# CatBoost modelinden elde edilen ortalama feature importance değerleri (user_id_encoded olmadan)
feature_importance_no_userid = feature_importances_catboost_no_userid['average']

# Feature importance değerlerini sırala
sorted_idx_no_userid = feature_importance_no_userid.sort_values(ascending=False).index
sorted_importance_no_userid = feature_importance_no_userid[sorted_idx_no_userid]

# Görselleştirmeyi yap
plt.figure(figsize=(10, 8))
plt.barh(range(len(sorted_idx_no_userid)), sorted_importance_no_userid, align='center')
plt.yticks(range(len(sorted_idx_no_userid)), sorted_idx_no_userid)
plt.xlabel('Average Feature Importance (Adversarial - No User ID)')
plt.title('Adversarial Model Feature Importance (No User ID)')
plt.gca().invert_yaxis() # En önemli özelliği en üste almak için y eksenini ters çevir

plt.tight_layout()
plt.show()

print("✅ Feature importance görselleştirme tamamlandı!")

# 1. VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN) - user_id_encoded ve day_cos çıkarıldı

In [ ]:
# ===============================================================================
# 1. VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN) - user_id_encoded ve day_cos çıkarıldı
# ===============================================================================

print("\n🔄 Adversarial validation için veri hazırlanıyor (user_id_encoded ve day_cos çıkarıldı)...")

# Train ve test veri setlerine kaynak etiketi ekle
X_train_modified_2 = X_train.copy() # Kopya al
X_test_aligned_modified_2 = X_test_aligned.copy() # Kopya al

# user_id_encoded ve day_cos sütunlarını çıkar
cols_to_drop = ['user_id_encoded', 'day_cos']
for col in cols_to_drop:
    if col in X_train_modified_2.columns:
        X_train_modified_2 = X_train_modified_2.drop(col, axis=1)
        print(f"   '{col}' train özelliklerinden çıkarıldı.")
    if col in X_test_aligned_modified_2.columns:
        X_test_aligned_modified_2 = X_test_aligned_modified_2.drop(col, axis=1)
        print(f"   '{col}' test özelliklerinden çıkarıldı.")


X_train_modified_2['source'] = 0  # 0 for train
X_test_aligned_modified_2['source'] = 1 # 1 for test

# Train ve test özelliklerini birleştir
# Kategorik sütunları birleştirmeden önce object'e çevir
for col in X_train_modified_2.columns:
    if X_train_modified_2[col].dtype == 'category':
        X_train_modified_2[col] = X_train_modified_2[col].astype(object)
        X_test_aligned_modified_2[col] = X_test_aligned_modified_2[col].astype(object)


adversarial_data_no_userid_daycos = pd.concat([X_train_modified_2, X_test_aligned_modified_2], ignore_index=True)

# Hedef değişkeni (source) ayır
adversarial_target_no_userid_daycos = adversarial_data_no_userid_daycos['source']
adversarial_features_no_userid_daycos = adversarial_data_no_userid_daycos.drop('source', axis=1)

print(f"   Birleştirilmiş veri boyutu (user_id_encoded ve day_cos olmadan): {adversarial_data_no_userid_daycos.shape}")
print("✅ Veri hazırlığı tamamlandı!")

# 2.ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE - user_id_encoded ve day_cos olmadan)

In [ ]:
# ===============================================================================
# 2. ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE - user_id_encoded ve day_cos olmadan)
# ===============================================================================

print("\n adversarial model eğitiliyor (CatBoost - user_id_encoded ve day_cos olmadan)...")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# Adversarial model için CatBoost sınıflandırıcısı - Parametreler aynı kalabilir
cat_adv_params_no_userid_daycos = {
    'objective': 'Logloss',  # Binary classification objective
    'eval_metric': 'AUC',
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'random_seed': 42,
    'verbose': 0, # Daha fazla bilgi için 100 yapabilirsiniz
    'task_type': 'GPU' if 'GPU' in str(globals().get('device', '')) else 'CPU',
    'early_stopping_rounds': 100
}

# Kategorik özellikleri belirle (user_id_encoded ve day_cos olmadan)
# CatBoost string/object türündeki kategoriklere doğrudan destek verir.
categorical_features_adv_no_userid_daycos = [col for col in adversarial_features_no_userid_daycos.columns if adversarial_features_no_userid_daycos[col].dtype == 'object' or adversarial_features_no_userid_daycos[col].dtype == 'category']

print(f"   Adversarial model için kategorik sütunlar (user_id_encoded ve day_cos olmadan): {categorical_features_adv_no_userid_daycos}")


# Çapraz doğrulama ile eğit
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds_catboost_no_userid_daycos = np.zeros(adversarial_data_no_userid_daycos.shape[0])
feature_importances_catboost_no_userid_daycos = pd.DataFrame(index=adversarial_features_no_userid_daycos.columns)

for fold, (train_idx, val_idx) in enumerate(kf.split(adversarial_features_no_userid_daycos, adversarial_target_no_userid_daycos)):
    print(f"\n   Fold {fold+1}/5")
    X_train_fold, X_val_fold = adversarial_features_no_userid_daycos.iloc[train_idx], adversarial_features_no_userid_daycos.iloc[val_idx]
    y_train_fold, y_val_fold = adversarial_target_no_userid_daycos.iloc[train_idx], adversarial_target_no_userid_daycos.iloc[val_idx]

    model = CatBoostClassifier(**cat_adv_params_no_userid_daycos)

    model.fit(X_train_fold, y_train_fold,
              eval_set=[(X_val_fold, y_val_fold)],
              cat_features=categorical_features_adv_no_userid_daycos,
              early_stopping_rounds=cat_adv_params_no_userid_daycos['early_stopping_rounds'],
              verbose=cat_adv_params_no_userid_daycos['verbose'])

    oof_preds_catboost_no_userid_daycos[val_idx] = model.predict_proba(X_val_fold)[:, 1]
    feature_importances_catboost_no_userid_daycos[f'fold_{fold+1}'] = model.get_feature_importance() # CatBoost feature importance alma

# OOF tahminleri üzerinden AUC hesapla
overall_auc_catboost_no_userid_daycos = roc_auc_score(adversarial_target_no_userid_daycos, oof_preds_catboost_no_userid_daycos)

print(f"\n✅ Adversarial model eğitimi tamamlandı!")
print(f"   Overall Adversarial AUC (CatBoost - user_id_encoded ve day_cos olmadan): {overall_auc_catboost_no_userid_daycos:.6f}")

# Feature importances ortalamasını hesapla
feature_importances_catboost_no_userid_daycos['average'] = feature_importances_catboost_no_userid_daycos.mean(axis=1)
feature_importances_catboost_no_userid_daycos = feature_importances_catboost_no_userid_daycos.sort_values('average', ascending=False)

print("\n   Önemli Özellikler (En Yüksekten En Düşüğe - CatBoost - user_id_encoded ve day_cos olmadan):")
print(feature_importances_catboost_no_userid_daycos[['average']].head(20)) # En önemli 20 özelliği göster

# 1.VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN) - user_id_encoded, day_cos, day_of_week ve day_sin çıkarıldı

In [ ]:
# ===============================================================================
# 1. VERİ HAZIRLIĞI (ADVERSARIAL VALIDATION İÇİN) - user_id_encoded, day_cos, day_of_week ve day_sin çıkarıldı
# ===============================================================================

print("\n🔄 Adversarial validation için veri hazırlanıyor (user_id_encoded, day_cos, day_of_week ve day_sin çıkarıldı)...")

# Train ve test veri setlerine kaynak etiketi ekle
X_train_modified_3 = X_train.copy() # Kopya al
X_test_aligned_modified_3 = X_test_aligned.copy() # Kopya al

# user_id_encoded, day_cos, day_of_week ve day_sin sütunlarını çıkar
cols_to_drop = ['user_id_encoded', 'day_cos', 'day_of_week', 'day_sin']
for col in cols_to_drop:
    if col in X_train_modified_3.columns:
        X_train_modified_3 = X_train_modified_3.drop(col, axis=1)
        print(f"   '{col}' train özelliklerinden çıkarıldı.")
    if col in X_test_aligned_modified_3.columns:
        X_test_aligned_modified_3 = X_test_aligned_modified_3.drop(col, axis=1)
        print(f"   '{col}' test özelliklerinden çıkarıldı.")


X_train_modified_3['source'] = 0  # 0 for train
X_test_aligned_modified_3['source'] = 1 # 1 for test

# Train ve test özelliklerini birleştir
# Kategorik sütunları birleştirmeden önce object'e çevir
for col in X_train_modified_3.columns:
    if X_train_modified_3[col].dtype == 'category':
        X_train_modified_3[col] = X_train_modified_3[col].astype(object)
        X_test_aligned_modified_3[col] = X_test_aligned_modified_3[col].astype(object)


adversarial_data_no_userid_daycos_dayofweek_daysin = pd.concat([X_train_modified_3, X_test_aligned_modified_3], ignore_index=True)

# Hedef değişkeni (source) ayır
adversarial_target_no_userid_daycos_dayofweek_daysin = adversarial_data_no_userid_daycos_dayofweek_daysin['source']
adversarial_features_no_userid_daycos_dayofweek_daysin = adversarial_data_no_userid_daycos_dayofweek_daysin.drop('source', axis=1)

print(f"   Birleştirilmiş veri boyutu (user_id_encoded, day_cos, day_of_week ve day_sin olmadan): {adversarial_data_no_userid_daycos_dayofweek_daysin.shape}")
print("✅ Veri hazırlığı tamamlandı!")

# 2.ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE - user_id_encoded, day_cos, day_of_week ve day_sin olmadan)

In [ ]:
# ===============================================================================
# 2. ADVERSARIAL MODEL EĞİTİMİ (CATBOOST İLE - user_id_encoded, day_cos, day_of_week ve day_sin olmadan)
# ===============================================================================

print("\n adversarial model eğitiliyor (CatBoost - user_id_encoded, day_cos, day_of_week ve day_sin olmadan)...")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# Adversarial model için CatBoost sınıflandırıcısı - Parametreler aynı kalabilir
cat_adv_params_no_userid_daycos_dayofweek_daysin = {
    'objective': 'Logloss',  # Binary classification objective
    'eval_metric': 'AUC',
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'random_seed': 42,
    'verbose': 0, # Daha fazla bilgi için 100 yapabilirsiniz
    'task_type': 'GPU' if 'GPU' in str(globals().get('device', '')) else 'CPU',
    'early_stopping_rounds': 100
}

# Kategorik özellikleri belirle (user_id_encoded, day_cos, day_of_week ve day_sin olmadan)
# CatBoost string/object türündeki kategoriklere doğrudan destek verir.
categorical_features_adv_no_userid_daycos_dayofweek_daysin = [col for col in adversarial_features_no_userid_daycos_dayofweek_daysin.columns if adversarial_features_no_userid_daycos_dayofweek_daysin[col].dtype == 'object' or adversarial_features_no_userid_daycos_dayofweek_daysin[col].dtype == 'category']

print(f"   Adversarial model için kategorik sütunlar (user_id_encoded, day_cos, day_of_week ve day_sin olmadan): {categorical_features_adv_no_userid_daycos_dayofweek_daysin}")


# Çapraz doğrulama ile eğit
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds_catboost_no_userid_daycos_dayofweek_daysin = np.zeros(adversarial_data_no_userid_daycos_dayofweek_daysin.shape[0])
feature_importances_catboost_no_userid_daycos_dayofweek_daysin = pd.DataFrame(index=adversarial_features_no_userid_daycos_dayofweek_daysin.columns)

for fold, (train_idx, val_idx) in enumerate(kf.split(adversarial_features_no_userid_daycos_dayofweek_daysin, adversarial_target_no_userid_daycos_dayofweek_daysin)):
    print(f"\n   Fold {fold+1}/5")
    X_train_fold, X_val_fold = adversarial_features_no_userid_daycos_dayofweek_daysin.iloc[train_idx], adversarial_features_no_userid_daycos_dayofweek_daysin.iloc[val_idx]
    y_train_fold, y_val_fold = adversarial_target_no_userid_daycos_dayofweek_daysin.iloc[train_idx], adversarial_target_no_userid_daycos_dayofweek_daysin.iloc[val_idx]

    model = CatBoostClassifier(**cat_adv_params_no_userid_daycos_dayofweek_daysin)

    model.fit(X_train_fold, y_train_fold,
              eval_set=[(X_val_fold, y_val_fold)],
              cat_features=categorical_features_adv_no_userid_daycos_dayofweek_daysin,
              early_stopping_rounds=cat_adv_params_no_userid_daycos_dayofweek_daysin['early_stopping_rounds'],
              verbose=cat_adv_params_no_userid_daycos_dayofweek_daysin['verbose'])

    oof_preds_catboost_no_userid_daycos_dayofweek_daysin[val_idx] = model.predict_proba(X_val_fold)[:, 1]
    feature_importances_catboost_no_userid_daycos_dayofweek_daysin[f'fold_{fold+1}'] = model.get_feature_importance() # CatBoost feature importance alma

# OOF tahminleri üzerinden AUC hesapla
overall_auc_catboost_no_userid_daycos_dayofweek_daysin = roc_auc_score(adversarial_target_no_userid_daycos_dayofweek_daysin, oof_preds_catboost_no_userid_daycos_dayofweek_daysin)

print(f"\n✅ Adversarial model eğitimi tamamlandı!")
print(f"   Overall Adversarial AUC (CatBoost - user_id_encoded, day_cos, day_of_week ve day_sin olmadan): {overall_auc_catboost_no_userid_daycos_dayofweek_daysin:.6f}")

# Feature importances ortalamasını hesapla
feature_importances_catboost_no_userid_daycos_dayofweek_daysin['average'] = feature_importances_catboost_no_userid_daycos_dayofweek_daysin.mean(axis=1)
feature_importances_catboost_no_userid_daycos_dayofweek_daysin = feature_importances_catboost_no_userid_daycos_dayofweek_daysin.sort_values('average', ascending=False)

print("\n   Önemli Özellikler (En Yüksekten En Düşüğe - CatBoost - user_id_encoded, day_cos, day_of_week ve day_sin olmadan):")
print(feature_importances_catboost_no_userid_daycos_dayofweek_daysin[['average']].head(20)) # En önemli 20 özelliği göster

# 4.ADVERSARIAL MODEL FEATURE IMPORTANCE GÖRSELLEŞTİRME (USER_ID_ENCODED, DAY_COS, DAY_OF_WEEK VE DAY_SIN OLMADAN)

In [ ]:
# ===============================================================================
# 4. ADVERSARIAL MODEL FEATURE IMPORTANCE GÖRSELLEŞTİRME (USER_ID_ENCODED, DAY_COS, DAY_OF_WEEK VE DAY_SIN OLMADAN)
# ===============================================================================

print("\n📊 Adversarial model feature importance görselleştiriliyor (user_id_encoded, day_cos, day_of_week ve day_sin olmadan)...")

# CatBoost modelinden elde edilen ortalama feature importance değerleri (çıkarılan sütunlar olmadan)
feature_importance_final = feature_importances_catboost_no_userid_daycos_dayofweek_daysin['average']

# Feature importance değerlerini sırala
sorted_idx_final = feature_importance_final.sort_values(ascending=False).index
sorted_importance_final = feature_importance_final[sorted_idx_final]

# Görselleştirmeyi yap
plt.figure(figsize=(10, 8))
plt.barh(range(len(sorted_idx_final)), sorted_importance_final, align='center')
plt.yticks(range(len(sorted_idx_final)), sorted_idx_final)
plt.xlabel('Average Feature Importance (Adversarial - Less Features)')
plt.title('Adversarial Model Feature Importance (Less Features)')
plt.gca().invert_yaxis() # En önemli özelliği en üste almak için y eksenini ters çevir

plt.tight_layout()
plt.show()

print("✅ Feature importance görselleştirme tamamlandı!")

In [ ]:
# ===============================================================================
# 6. FINAL MODEL EĞİTİMİ (FULL DATA - AZALTILMIŞ ÖZELLİK SETİ)
# ===============================================================================

print("\n🏆 Final model eğitimi başlatılıyor (azaltılmış özellik seti)...")

final_models_reduced = {}

# Veriyi hazırla (Adversarial validation sonrası kalan özellikler)
# X_train_modified_3 ve X_test_aligned_modified_3, user_id_encoded, day_cos, day_of_week, day_sin olmadan hazırlanmıştı
# Target (y_train_log) hala orijinal train setinden geliyor

y_train_final = y_train_log # Log dönüşümlü target
X_train_final = X_train_modified_3.drop('source', axis=1) # Kaynak sütununu çıkar
X_test_final = X_test_aligned_modified_3.drop('source', axis=1) # Kaynak sütununu çıkar

# Sütunların uyumlu olduğunu kontrol et (adversarial features ile aynı sütunlar olmalı)
if not X_train_final.columns.equals(X_test_final.columns):
    print("   ⚠️ Train ve test özellikleri eşleşmiyor! Sütunları hizalanıyor.")
    X_test_final = X_test_final.reindex(columns=X_train_final.columns, fill_value=0) # Fill_value 0 veya uygun bir değer olabilir

print(f"   📊 X_train (Final): {X_train_final.shape}, X_test (Final): {X_test_final.shape}")


# Kategorik sütunları belirle (artık user_id_encoded yok)
categorical_features_final = [col for col in X_train_final.columns if X_train_final[col].dtype == 'object' or X_train_final[col].dtype == 'category']
# duration_segment hala category olabilir, object'e çevirdiğimiz için burada object olarak görünebilir.

print(f"   📊 Final model için kategorik sütunlar: {categorical_features_final}")


# Modelleri Eğit

# CatBoost final (azaltılmış özellik seti ile)
print("\n   🐱 CatBoost final eğitiliyor...")
# CatBoost params are defined earlier as cat_params
final_models_reduced['catboost'] = CatBoostRegressor(**cat_params)
final_models_reduced['catboost'].fit(
    X_train_final,
    y_train_final,
    cat_features=categorical_features_final,
    verbose=0
)

# LightGBM final (azaltılmış özellik seti ile)
print("\n   💡 LightGBM final eğitiliyor...")
# LightGBM params are defined earlier as lgb_params
final_models_reduced['lightgbm'] = lgb.LGBMRegressor(**lgb_params)

# LightGBM kategorik özellikler için indeksleri belirle
lgb_categorical_indices_final = [X_train_final.columns.get_loc(col) for col in categorical_features_final]

# LightGBM için kategorik sütunları int'e çevir
X_train_lgb_final = X_train_final.copy()
X_test_lgb_final = X_test_final.copy()

for col in categorical_features_final:
     if X_train_lgb_final[col].dtype == 'object' or X_train_lgb_final[col].dtype == 'category':
            X_train_lgb_final[col] = X_train_lgb_final[col].astype(str).astype('category').cat.codes
            X_test_lgb_final[col] = X_test_lgb_final[col].astype(str).astype('category').cat.codes


final_models_reduced['lightgbm'].fit(
    X_train_lgb_final,
    y_train_final,
    categorical_feature=lgb_categorical_indices_final
    # Early stopping callbacks might not be needed for final training on full data
)

print("✅ Final model eğitimi tamamlandı!")

# Model performanslarını kontrol et (Train RMSE - Sadece bilgi amaçlı)
print("\n📈 Final model performansları (Train RMSE - Azaltılmış Özellik Seti):")
for name, model in final_models_reduced.items():
    if name == 'lightgbm': # LightGBM needs integer-encoded categoricals
         train_pred = model.predict(X_train_lgb_final)
    else: # CatBoost can handle original types
         train_pred = model.predict(X_train_final)
    train_rmse = np.sqrt(mean_squared_error(y_train_final, train_pred))
    print(f"{name.upper()}: Train RMSE = {train_rmse:.4f}")

In [ ]:
# ===============================================================================
# 7. TAHMİN VE SUBMİSSİON (AZALTILMIŞ ÖZELLİK SETİ)
# ===============================================================================

print("\n🔮 Tahminler yapılıyor (azaltılmış özellik seti)...")

# Test predictions (azaltılmış özellik seti ile)
test_preds_reduced = {}
for name, model in final_models_reduced.items():
     if name == 'lightgbm': # LightGBM needs integer-encoded categoricals
         test_preds_reduced[name] = model.predict(X_test_lgb_final)
     else: # CatBoost can handle original types
         test_preds_reduced[name] = model.predict(X_test_final)


# Ensemble prediction (azaltılmış özellik seti için ağırlıkları tekrar hesapla veya öncekileri kullan)
# Basitlik için önceki ağırlıkları kullanalım, veya val seti oluşturup yeniden hesaplayabiliriz.
# Önceki ağırlıklar eğitimdeki performanslarına dayanıyordu, şimdi özellik seti değişti.
# Daha doğru bir yaklaşım, azaltılmış özellik seti ile bir val seti üzerinde modelleri değerlendirip ağırlıkları yeniden hesaplamak olur.
# Ancak hızlıca denemek için mevcut ağırlıkları kullanalım.

# Ağırlıklar önceki eğitimden: weights = {'catboost': 0.337, 'lightgbm': 0.332, 'xgboost': 0.330}
# Şimdi 2 modelimiz var, sadece catboost ve lightgbm ağırlıklarını orantılayalım
total_weight_cat_lgb = weights['catboost'] + weights['lightgbm']
weights_reduced = {
    'catboost': weights['catboost'] / total_weight_cat_lgb,
    'lightgbm': weights['lightgbm'] / total_weight_cat_lgb
}

print(f"   📊 Azaltılmış Özellik Seti Ağırlıkları:")
print(f"      CatBoost: {weights_reduced['catboost']:.3f}")
print(f"      LightGBM: {weights_reduced['lightgbm']:.3f}")


final_prediction_log_reduced = (weights_reduced['catboost'] * test_preds_reduced['catboost'] +
                               weights_reduced['lightgbm'] * test_preds_reduced['lightgbm'])


# Convert back to original scale
final_prediction_reduced = np.expm1(final_prediction_log_reduced)
final_prediction_reduced[final_prediction_reduced < 0] = 0 # ensure no negative predictions


# Create model predictions DataFrame
model_predictions_df_reduced = pd.DataFrame({
    'user_session': test_features_df['user_session'], # Use original test sessions to map back
    'session_value': final_prediction_reduced
})


# ===============================================================================
# 8. SUBMİSSİON DOSYASI OLUŞTURMA (AZALTILMIŞ ÖZELLİK SETİ)
# ===============================================================================

print("\n📝 Submission dosyası oluşturuluyor (azaltılmış özellik seti)...")

# Official template
submission_df_reduced = test[['user_session']].drop_duplicates().copy()

# Fill leaked data first (leak_map from step 2)
if len(leak_map) > 0:
    submission_df_reduced['session_value'] = submission_df_reduced['user_session'].map(leak_map)
    leaked_count = len(submission_df_reduced) - submission_df_reduced['session_value'].isnull().sum()
    print(f"   🔍 {leaked_count} leaked prediction eklendi")
else:
    submission_df_reduced['session_value'] = np.nan
    leaked_count = 0

# Fill remaining with model predictions
model_predictions_map_reduced = model_predictions_df_reduced.set_index('user_session')['session_value']
submission_df_reduced['session_value'] = submission_df_reduced['session_value'].fillna(
    submission_df_reduced['user_session'].map(model_predictions_map_reduced)
)

filled_by_model = len(submission_df_reduced) - submission_df_reduced['session_value'].isnull().sum() - leaked_count
print(f"   🤖 {filled_by_model} model prediction eklendi")


# Final check for any remaining NaNs (should not happen if test_unseen_df covers all non-leaked)
if submission_df_reduced['session_value'].isnull().sum() > 0:
    print("   ⚠️ Boş değerler ortalama ile dolduruluyor...")
    submission_df_reduced['session_value'].fillna(y_train.mean(), inplace=True)


# Save submission
submission_path_reduced = '/content/submission_reduced_features.csv'
submission_df_reduced.to_csv(submission_path_reduced, index=False)

print(f"🎉 BAŞARILI! Submission dosyası hazır: {submission_path_reduced}")
print(f"📊 Toplam prediction: {len(submission_df_reduced)}")
# Note: Ensemble RMSE here would require re-calculating on a validation set with reduced features
# print(f"📈 Ensemble validation RMSE: {ensemble_rmse:.6f}") # This is from the previous run, not representative of reduced features
print("\n" + "="*60)
print("🏁 Azaltılmış özellik seti ile model pipeline tamamlandı!")
print("="*60)